### Importación de librerías a utilizar

In [1]:
import numpy as np

### Función de polling table - Tabla de sondeo para ONUs

In [4]:
def create_polling_table(onu_no, onu_olt_dist, grant_size,link_speed):
    c = 2*1e5
    # Inicialización de tabla con sondeos
    polling_table = np.zeros((onu_no,4))
    
    #indice de ONU
    polling_table[:, 0] = np.arange(1, onu_no + 1)

    # Round Time Trip (RTT)
    polling_table[:, 1] = 2 * onu_olt_dist / c

    # Tiempo de Transmisión
    polling_table[:, 2] = grant_size * 8 / link_speed

    # Columna 4: Tiempo total (RTT + tiempo de transmisión).
    polling_table[:, 3] = polling_table[:, 1] + polling_table[:, 2]

    return polling_table